**NARGEOT** Melvine
**PATTE** Mathys

# Projet Outil Informatique

https://www.sciencedirect.com/topics/earth-and-planetary-sciences/volcanic-aerosol

The Greenhouse Effect, Aerosols, and Climate Change

Daniel Kirk-Davidoff, in Green Chemistry, 2018

"Volcanoes emit sulfur dioxide gas (SO2), which reacts with water in the atmosphere to form sulfuric acid (H2SO4). When volcanic plumes are emitted powerfully enough to reach the stratosphere,a the H2SO4 can form a persistent haze of liquid droplets, reflecting away sunlight and cooling the earth for a year or two."

In [ ]:
%pip install numpy matplotlib pandas # Handle data
%pip install xarray netcdf4 h5netcdf # Handle NetCFD file
%pip install folium # World map
%pip install beautifulsoup4 # Web scrapping
%pip install geopandas
%pip install slugify

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import folium as fl
from bs4 import BeautifulSoup
from urllib.request import urlopen
import geopandas as gpd
import slugify

## Données

### Sources

https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/volcanoes?page=1

https://volcano.si.edu/search_eruption.cfm

https://www.kaggle.com/datasets/jessemostipak/volcano-eruptions/

https://data.cerdi.uca.fr/erup-vol/

https://public.opendatasoft.com/explore/dataset/significant-volcanic-eruption-database/export/

https://www.ngdc.noaa.gov/hazel/view/hazards/volcano/event-search/

https://berkeleyearth.org/data/

https://disc.gsfc.nasa.gov/datasets?keywords=%22msvolso2l4%22%7C%22tomsn7so2%22%7C%22OMPS_NPP_NMSO2_PCA_L2%22%7C%22MSAQSO2L4%22&page=1&subject=Aerosols

https://measures.gesdisc.eosdis.nasa.gov/data/SO2/MSVOLSO2L4.4/MSVOLSO2L4_v04-00-2022m0505.txt

### Import

In [ ]:
from urllib.request import urlopen
from io import StringIO

# https://measures.gesdisc.eosdis.nasa.gov/data/SO2/MSVOLSO2L4.4/MSVOLSO2L4_v04-00-2022m0505.txt
data = urlopen("https://melvin-klein.github.io/outinfo/so2.txt").readlines()[47:]

# Some lines have too many cells, we delete them.
data[1172] = data[1172][:-4] + b'\n'
data[2033] = data[2033][:-4] + b'\n'

data = str(b''.join(data), 'utf-8')

data = StringIO(data) 

so2 = pd.read_csv(data, sep="\t")

so2

In [ ]:
columns = ['volcano', 'lat', 'lon', 'yyyy', 'mm', 'so2(kt)']

so2_cleared = so2.dropna(subset=columns)

so2_cleared = so2_cleared[columns]

so2_cleared

In [ ]:
# Add countries to volcano

gdf_eruptions = gpd.read_file("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/significant-volcanic-eruption-database/exports/geojson?lang=fr&timezone=Europe%2FBerlin")

columns = ['name', 'country', 'elevation', 'type', 'year', 'month', 'geometry']

gdf_eruptions_cleared = gdf_eruptions[columns]

gdf_eruptions_cleared = gdf_eruptions_cleared.dropna(subset=columns)

gdf_eruptions_cleared

## Try to merge the two Geopandas dataframe

In [ ]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(so2_cleared['lon'], so2_cleared['lat'])]
gdf_so2_cleared = gpd.GeoDataFrame(so2_cleared, geometry=geometry, crs='EPSG:4326')
gdf_so2_cleared

In [ ]:
joined_gdf = gpd.sjoin(gdf_so2_cleared, gdf_eruptions_cleared, how='left', op='intersects')
filtered_gdf = joined_gdf[joined_gdf['yyyy'] == joined_gdf['year']]
filtered_gdf

## End of try

In [ ]:
volcano_weather = pd.DataFrame(columns=['year', 'month', 'volcano_name', 'country', 'eruption_so2', 'lat', 'lng'])

In [ ]:
volcano_weather['year'] = pd.to_datetime(so2_cleared['yyyy'], format="%Y").dt.year
volcano_weather['month'] = pd.to_datetime(so2_cleared['mm'], format="%m").dt.month
volcano_weather[['volcano_name', 'eruption_so2', 'lat', 'lng']] = so2_cleared[['volcano', 'so2(kt)', 'lat', 'lon']]

volcano_weather['id'] = volcano_weather['volcano_name'].apply(lambda x: slugify.slugify(x))

volcano_id_customdf = sorted(volcano_weather[volcano_weather['id'].isin(volcano_id_geojson)]['id'].unique().tolist())

len(volcano_weather['volcano_name'].unique().tolist())

volcano_weather

In [ ]:
volcano_id_geojson

In [ ]:
set(volcano_weather['id'].unique().tolist()) - set(volcano_id_geojson)

In [ ]:
volcano_map = fl.Map(zoom_start=10)

for index, row in volcano_weather.drop_duplicates(subset='volcano_name', keep='first').iterrows():
    fl.Marker(
        location=[row['lat'], row['lng']],
        popup=row['volcano_name']
    ).add_to(volcano_map)

volcano_map

In [ ]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3_en.php?block_no=47401"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [ ]:
rows = soup.find_all("tr", {"class": "mtx"})

for row in rows:
    data = row.get_text().split(' ')

rows[1].get_text(" ").split(' ')

## Analyse

In [ ]:
monthly_t = pd.read_json("https://pkgstore.datahub.io/core/global-temp/monthly_json/data/4c7af7363a20648a68b8f2038a6765d6/monthly_json.json") # Reading the json file of global monthly temperature anomaly

monthly_t = monthly_t.loc[monthly_t["Source"] == "GISTEMP"] # Only keep values from GISTEMP (nasa)
monthly_t.reset_index(inplace=True) # Reset row index to manipulate rows easily

# Splitting the date in format yyyy-mm-dd into Year and Month to use it with other data later
year = [monthly_t["Date"][date].year for date in range(len(monthly_t["Date"]))] # Create a 'Year' column from the timestamp given in 'Date'
monthly_t["year"] = year

month = [monthly_t["Date"][date].month for date in range(len(monthly_t["Date"]))] # Create a 'Month' column from the timestamp given in 'Date'
monthly_t["month"] = month

monthly_t = monthly_t.drop(['Date', 'Source'], axis=1) # Delete useless rows for later use
monthly_t = monthly_t.reindex(columns=['year', 'month', 'Mean']) # Put date first then temperature anomaly
monthly_t.sort_values(by=['year','month'], inplace=True) # Sort by ascending date
monthly_t.reset_index(inplace=True, drop=True) # Reset index for readability 
monthly_t = monthly_t.rename(columns={"Mean" : "t_anomaly"}) # rename Mean for readability
monthly_t


In [ ]:
monthly_t.reset_index(inplace=True, drop=True)
volcano_weather.reset_index(inplace=True, drop=True)

date = [str(volcano_weather["year"][date]) + str(volcano_weather["month"][date]) for date in range(len(volcano_weather["year"]))]
volcano_weather["date"] = date
date = [str(monthly_t["year"][date]) + str(monthly_t["month"][date]) for date in range(len(monthly_t["year"]))]
monthly_t["Date"] = date

monthly_t = monthly_t[monthly_t['Date'].isin(volcano_weather["date"])]


volcano_temperature = pd.merge(monthly_t, volcano_weather, on=["year", "month"], how="right")
print(volcano_temperature)


In [ ]:
plt.figure(figsize=(9,4), dpi=100)
plt.plot(volcano_temperature["year"], volcano_temperature["t_anomaly"])
plt.xlabel('Year')
plt.ylabel('Temperature anomaly')
plt.legend(['Anomaly of temperature for each year'], loc=(0.02, 0.88))
plt.title('Comparison of temperature and sulfure emission', pad=10, fontsize=14, style='italic')
plt.subplot()
plt.plot(volcano_temperature["year"], volcano_temperature["eruption_so2"]/10000); plt.legend("Sulfure emission for each year")

In [ ]:
plt.figure(figsize=(8,4), dpi=100)
plt.plot(volcano_temperature["t_anomaly"], np.log(volcano_temperature["eruption_so2"]))
plt.xlabel('temperature anomaly'); plt.ylabel('sulfure emission'); # labels
plt.title('Visualisation de données éparses', pad=10, fontsize=14, style='italic'); # titre